In [13]:
from statsmodels.sandbox.regression.gmm import IV2SLS
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = pd.read_csv('fin_dataset_africa copy.csv', delimiter=';')
data = data.replace('..', np.nan)
data[data.columns[2:]] = data[data.columns[2:]].apply(pd.to_numeric, errors='coerce')
data.fillna(data.mean(), inplace=True)

df = data.copy()

# remove all nan values
df = df.dropna()

# remove all .. values
df = df.replace('..', np.nan)


# Instrumental variables
df['lag_debt'] = df.groupby('Country_Name')['Government_Debt_Percent_of_GDP'].shift(1)
# remove the first year for each country
df = df.dropna()


# Define your variables
endog = df['GDP_per_capita_growth_annual_Percent']

exog = sm.add_constant(df[['Gross_capital_formation_Percent_of_GDP',
                           'Gross_national_expenditure_Percent_of_GDP',
                           'Net_barter_terms_of_trade_index_2015_100',
                           'Population_growth_annual_Percent',
                           'Trade_Percent_of_GDP']])

instruments = sm.add_constant(df[['lag_debt', 'Gross_capital_formation_Percent_of_GDP',
                                  'Gross_national_expenditure_Percent_of_GDP',
                                  'Net_barter_terms_of_trade_index_2015_100',
                                  'Population_growth_annual_Percent',
                                  'Trade_Percent_of_GDP']])

endog_regressor = df['Government_Debt_Percent_of_GDP']

# Combine exogenous and endogenous regressors for the IV2SLS model
exog_iv = exog.copy()
exog_iv['Government_Debt_Percent_of_GDP'] = endog_regressor

# Running 2SLS
iv_model = IV2SLS(endog, exog_iv, instruments).fit()

# Print the results from IV2SLS
print(iv_model.summary())


                                   IV2SLS Regression Results                                    
Dep. Variable:     GDP_per_capita_growth_annual_Percent   R-squared:                       0.014
Model:                                           IV2SLS   Adj. R-squared:                  0.008
Method:                                       Two Stage   F-statistic:                     1.920
                                          Least Squares   Prob (F-statistic):             0.0750
Date:                                  Thu, 18 Apr 2024                                         
Time:                                          12:21:45                                         
No. Observations:                                   864                                         
Df Residuals:                                       857                                         
Df Model:                                             6                                         
                              

/var/folders/zn/8wl2ftxd7sg0pjvzvx59s5xr0000gn/T/ipykernel_24803/759617557.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)
